(adc-hpc:label)=
# HPC-QC implementation

To enable large-scale ADC(2) calculations on HPC systems, some considerations need to be taken into account in the implementation. Conventionally, the molecular orbital (MO) integrals, which are essentially tensors with four dimensions, are computed by integral transformation on a single compute node (or workstation). This is unfortunately not practical on HPC clusters, since a cluster node usually has moderate amount of memory which is not enough to store an object with $N^4$ elements (here $N$ is the number of orbitals). Therefore, the MO integral needs to be stored in a distributed way; in other words, each node need to store a portion of the four-dimensional MO integral. It is then natural to distribute the computation of MO integral as well, such that each node only computes part of the integral that needs to be stored locally. This leads to the so-called Fock matrix-driver integral transformation approach, where MO integrals are expressed in terms of Fock-like matrices {cite}`Hohenstein2015, gator`

$$
\langle ij || kl \rangle = \sum_{\gamma \delta}^{N} \left [ C_{\gamma k} F_{\gamma \delta}^{ij} C_{\delta l} - C_{\gamma l} F_{\gamma \delta}^{ij} C_{\delta k} \right ] \, ,
$$

where

$$
F_{\gamma \delta}^{ij} &= \sum_{\alpha \beta}^{N} D_{\alpha \beta}^{ij} (\alpha \gamma|\delta \beta) \\
D_{\alpha \beta}^{ij} &= C_{\alpha i} C_{\beta j} \, .
$$

In practice the $ij$ pairs and the corresponding $kl$ matrices are stored on individual nodes. This also facilitates parallelization of subsequent $\sigma$-build (matrix-vector multiplication) with minimal amount of communications.

In the HPC-QC implementation of ADC(2), another object that cannot be stored on a cluster node is the excitation vector that involves two particle-hole pairs (i.e., the double excitation part). This is because the vector of double excitation has four indices and its size increases as $O(N^4)$. Fortunately, explicit storage of the double excitation vector can be avoided by converting the standard eigenproblem in single and double space to a nonlinear eigenproblem in single space {cite}`Hattig2006`. By expanding the ADC(2) equation

$$
\begin{pmatrix} \mathbf{M}_{\text{SS}} & \mathbf{M}_{\text{SD}} \\ \mathbf{M}_{\text{DS}} & \mathbf{M}_{\text{DD}} \end{pmatrix}
\begin{pmatrix} \mathbf{X}_{\text{S}} \\ \mathbf{X}_{\text{D}} \end{pmatrix}
= \omega \begin{pmatrix} \mathbf{X}_{\text{S}} \\ \mathbf{X}_{\text{D}} \end{pmatrix} \, ,
$$

we have

$$
\mathbf{M}_{\text{SS}} \mathbf{X}_{\text{S}} + \mathbf{M}_{\text{SD}} \mathbf{X}_{\text{D}} = \omega \mathbf{X}_{\text{S}}
$$
$$
\mathbf{M}_{\text{DS}} \mathbf{X}_{\text{S}} + \mathbf{M}_{\text{DD}} \mathbf{X}_{\text{D}} = \omega \mathbf{X}_{\text{D}} \, ,
$$

where $\omega$ is the excitation energy, and $\mathbf{X}_{\text{S}}$ and $\mathbf{X}_{\text{D}}$ are the single and double excitation parts of the eigenvectors, respectively. The above two equations can be rewritten as (after eliminating $\mathbf{X}_{\text{D}}$)

$$
\left[ \mathbf{M}_{\text{SS}} + \mathbf{M}_{\text{SD}} (\omega - \mathbf{M}_{\text{DD}})^{-1} \mathbf{M}_{\text{DS}} \right] \mathbf{X}_{\text{S}} = \omega \mathbf{X}_{\text{S}} \, .
$$

Note that $\mathbf{M}_{\text{DD}}$ is diagonal in ADC(2). Let

$$
\mathbf{M}_{\text{eff}} = \mathbf{M}_{\text{SS}} + \mathbf{M}_{\text{SD}} (\omega - \mathbf{M}_{\text{DD}})^{-1} \mathbf{M}_{\text{DS}} \, ,
$$

we have

$$
\mathbf{M}_{\text{eff}} \mathbf{X}_{\text{S}} = \omega \mathbf{X}_{\text{S}} \, .
$$

This is a nonlinear eigenproblem since $\mathbf{M}_{\text{eff}}$ depends on $\omega$. The excitation energies and single excitation vectors can then be solved by a nonlinear eigensolver. Note that the resultant single excitation vectors need to be scaled to satisfy the following condition

$$
\mathbf{X}_{\text{S}}^\dagger \mathbf{X}_{\text{S}}^{ } + \mathbf{X}_{\text{D}}^\dagger \mathbf{X}_{\text{D}}^{ }  = 1 \, ,
$$

where $\mathbf{X}_{\text{D}}^\dagger \mathbf{X}_{\text{D}}$ can be calculated as

$$
\mathbf{X}_{\text{D}}^\dagger \mathbf{X}_{\text{D}} = \mathbf{X}_{\text{S}}^\dagger \mathbf{M}_{\text{SD}} (\omega - \mathbf{M}_{\text{DD}})^{-2} \mathbf{M}_{\text{DS}} \mathbf{X}_{\text{S}} \, .
$$
